# Estratégias de treino

Até agora no curso, vimos somente uma estratégia de treino para redes neurais: o treinamento do zero.
Entretanto, há outras formas de se explorar redes neurais.
Nessa aula, vamos rever a estratégia treinamento do zero além de apresentar duas novas formas:

1.   rede neural como um extrator de características, e
2.   *fine-tuning*.

Para cada uma dessas estratégias, vamos apresentar sua definição, vantagens e desvantagens.
Antes, vamos instalar o Pytorch, importar alguns pacotes e definir funções para carregar os dados.


In [1]:
!pip3 install torch torchvision

In [0]:
import time, os, sys, numpy as np
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F

from torch import optim
from torchsummary import summary


import time, os, sys, numpy as np

# Test if GPU is avaliable, if not, use cpu instead
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
n = torch.cuda.device_count()
devices_ids= list(range(n))

In [0]:
def load_data_cifar10(batch_size, resize=None, root=os.path.join(
        '~', '.pytorch', 'datasets', 'fashion-mnist')):
    """Download the Cifar10-MNIST dataset and then load into memory."""
    root = os.path.expanduser(root)
    transformer = []
    if resize:
        transformer += [torchvision.transforms.Resize(resize)]
    transformer += [torchvision.transforms.ToTensor()]
    transformer = torchvision.transforms.Compose(transformer)

    mnist_train = torchvision.datasets.CIFAR10(root=root, train=True,download=True,transform=transformer)
    mnist_test = torchvision.datasets.CIFAR10(root=root, train=False,download=True,transform=transformer)
    num_workers = 0 if sys.platform.startswith('win32') else 4



    train_iter = torch.utils.data.DataLoader(mnist_train,
                                  batch_size, shuffle=True,
                                  num_workers=num_workers)
    test_iter = torch.utils.data.DataLoader(mnist_test,
                                 batch_size, shuffle=False,
                                 num_workers=num_workers)
    return train_iter, test_iter

# funções básicas
def _get_batch(batch):
    """Return features and labels on ctx."""
    features, labels = batch
    if labels.type() != features.type():
        labels = labels.type(features.type())
    return (torch.nn.DataParallel(features, device_ids=devices_ids),
            torch.nn.DataParallel(labels, device_ids=devices_ids), features.shape[0])

# Função usada para calcular acurácia
def evaluate_accuracy(data_iter, net, loss):
    """Evaluate accuracy of a model on the given data set."""

    acc_sum, n, l = torch.Tensor([0]), 0, 0
    
    with torch.no_grad():
      for X, y in data_iter:
          #y = y.astype('float32')
          X, y = X.to(device), y.to(device)
          y_hat = net(X)
          l += loss(y_hat, y).sum()
          acc_sum += (y_hat.argmax(axis=1) == y).sum().item()
          n += y.size()[0]

    return acc_sum.item() / n, l.item() / len(data_iter)
  
# Função usada no treinamento e validação da rede
def train_validate(net, train_iter, test_iter, batch_size, trainer, loss,
                   num_epochs):
    print('training on', device)
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()
        for X, y in train_iter:
            X, y = X.to(device), y.to(device)
            y_hat = net(X)
            trainer.zero_grad()
            l = loss(y_hat, y).sum()
            l.backward()
            trainer.step()
            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(axis=1) == y).sum().item()
            n += y.size()[0]
        test_acc, test_loss = evaluate_accuracy(test_iter, net, loss)
        print('epoch %d, train loss %.4f, train acc %.3f, test loss %.4f, '
              'test acc %.3f, time %.1f sec'
              % (epoch + 1, train_l_sum / len(train_iter), train_acc_sum / n, test_loss, 
                 test_acc, time.time() - start))

## Treinamento do zero

Como dito anteriormente, essa foi a única estratégia vista até o momento no curso.
Nessa estratégia, uma rede neural é proposta, **inicializada com pesos aleatórios** e treinada até convergir.
A **vantagem** dessa estratégia é liberdade para definir como quiser a arquitetura da rede e seus hiper-parâmetros
Por outro lado, a **desvantagem** é que essa estratégia requer muitos dados para convergir a rede inicializada aleatoriamente.
Logo, se tivermos poucos dados, essa não é a estratégia mais recomendada.
Abaixo, uma representação visual dessa estratégia.

<p align="center">
  <img width=600 src="https://drive.google.com/uc?export=view&id=1_bBQjyoDqB3kQMncmVkuJwSxDs3rqUmM">
</p>

Apesar de já termos visto essa estratégia na prática, vamos vê-la aqui novamente para efeitos de comparação com as outras técnicas. Para tal, vamos, primeiro, definimos a arquitetura da [AlexNet](https://papers.nips.cc/paper/4824-imagenet-classification-with-deep-convolutional-neural-networks.pdf).



In [0]:
class AlexNet(nn.Module):
    r"""AlexNet model from the `"One weird trick..." `_ paper.

    Parameters
    ----------
    classes : int, default 10
        Number of classes for the output layer.
    """
    def __init__(self,input_channels, classes=10, **kwargs):
        super(AlexNet, self).__init__(**kwargs)
                
        self.conv1 = nn.Conv2d(in_channels=input_channels,kernel_size=11, out_channels=64, stride=4)             # entrada: (b, 3, 227, 227) e saida: (b, 64, 55, 55)
        self.max_pool1 = nn.MaxPool2d(kernel_size=3,stride=2)                                                    # entrada: (b, 64, 55, 55) e saida: (b, 64, 27, 27)
        self.conv2 = nn.Conv2d(in_channels=64,out_channels=192, kernel_size=5, padding=2)                        # entrada: (b, 64, 27, 27) e saida: (b, 192, 27, 27)
        self.max_pool2 = nn.MaxPool2d(kernel_size=3,stride=2)                                                    # entrada: (b, 192, 27, 27) e saida: (b, 192, 13, 13)
        self.conv3 = nn.Conv2d(in_channels=192,out_channels=384,kernel_size=3,padding=1)                         # entrada: (b, 192, 13, 13) e saida: (b, 384, 13, 13)
        self.conv4 = nn.Conv2d(in_channels=384,out_channels=256,kernel_size=3,padding=1)                         # entrada: (b, 384, 13, 13) e saida: (b, 256, 13, 13)
        self.conv5 = nn.Conv2d(in_channels=256,out_channels=256,kernel_size=3,padding=1)                         # entrada: (b, 256, 13, 13) e saida: (b, 256, 13, 13)
        self.max_pool3 = nn.MaxPool2d(kernel_size=3,stride=2)                                                    # entrada: (b, 256, 13, 13) e saida: (b, 256, 6, 6)
        self.flatten  = nn.Flatten()                                                                             # entrada: (b, 256, 13, 13) e saida: (b, 256*6*6) = (b, 9216)
        self.linear1 = nn.Linear(9216, 4096)                                                                     # entrada: (b, 9216) e saida: (b, 4096)
        self.dropout1 = nn.Dropout(0.5)
        self.linear2 = nn.Linear(4096, 4096)                                                                     # entrada: (b, 4096) e saida: (b, 4096)
        self.dropout2 = nn.Dropout(0.5)
        self.output = nn.Linear(4096, classes)                                                                   # entrada: (b, 4096) e saida: (b, 10)
          

    def forward(self, x):
        output_conv1 = F.relu(self.conv1(x))
        output_max_pool1 = self.max_pool1(output_conv1)
        output_conv2 = F.relu(self.conv2(output_max_pool1))
        output_max_pool2 = self.max_pool2(output_conv2)
        output_conv3 = F.relu(self.conv3(output_max_pool2))
        output_conv4 = F.relu(self.conv4(output_conv3))
        output_conv5 = F.relu(self.conv5(output_conv4))
        output_max_pool3 = self.max_pool3(output_conv5)
        output_flatten = self.flatten(output_max_pool3)
        output_linear1 = F.relu(self.linear1(output_flatten))
        output_dropout1 = self.dropout1(output_linear1)
        output_linear2 = F.relu(self.linear2(output_dropout1))
        output_dropout2 = self.dropout2(output_linear2)
        output = self.output(output_dropout2)

        return output


In [15]:
# parâmetros: número de epochs, learning rate (ou taxa de aprendizado), 
# tamanho do batch, e lambda do weight decay
num_epochs, lr, batch_size, wd_lambda = 20, 0.01, 100, 0.0001


# rede baseada na AlexNet-5 
net = AlexNet(3, 10)

# Sending model to device
net.to(device)
print(summary(net,(3,227,227))) # visualize number of parameters' net, output of each layer and total mega bytes necessary for forward pass
                                # and stored weights. 

# função de custo (ou loss)
loss = nn.CrossEntropyLoss()

# carregamento do dado: mnist
train_iter, test_iter = load_data_cifar10(batch_size, resize=227)

# trainer do gluon
trainer = optim.SGD(net.parameters(), lr=lr, weight_decay=wd_lambda, momentum=0.9)

# treinamento e validação via Pytorch
train_validate(net, train_iter, test_iter, batch_size, trainer, loss, 
                num_epochs)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 55, 55]          23,296
         MaxPool2d-2           [-1, 64, 27, 27]               0
            Conv2d-3          [-1, 192, 27, 27]         307,392
         MaxPool2d-4          [-1, 192, 13, 13]               0
            Conv2d-5          [-1, 384, 13, 13]         663,936
            Conv2d-6          [-1, 256, 13, 13]         884,992
            Conv2d-7          [-1, 256, 13, 13]         590,080
         MaxPool2d-8            [-1, 256, 6, 6]               0
           Flatten-9                 [-1, 9216]               0
           Linear-10                 [-1, 4096]      37,752,832
          Dropout-11                 [-1, 4096]               0
           Linear-12                 [-1, 4096]      16,781,312
          Dropout-13                 [-1, 4096]               0
           Linear-14                   

É muito comum se usar redes já existentes para aprender características em novos dados.
Por isso, muitos frameworks já deixam as arquiteturas mais famosas pré-implementadas para que possam ser usadas.

No Pytorch, podemos importar uma rede [AlexNet](https://papers.nips.cc/paper/4824-imagenet-classification-with-deep-convolutional-neural-networks.pdf) usando o pacote [torchvision.models](https://pytorch.org/docs/stable/torchvision/models.html#torchvision-models) do Pytorch.
Há várias arquiteturas pré-definidas nessa biblioteca, incluindo várias [DenseNets](https://arxiv.org/pdf/1608.06993.pdf) e [ResNets](https://arxiv.org/abs/1603.05027), [VGGs](https://arxiv.org/abs/1409.1556), [SqueezeNets](https://arxiv.org/abs/1602.07360), etc.

In [6]:
# parâmetros: número de epochs, learning rate (ou taxa de aprendizado), 
# tamanho do batch, e lambda do weight decay
num_epochs, lr, batch_size, wd_lambda = 20, 0.01, 100, 0.0001

# rede baseada na AlexNet-5 
net = torchvision.models.alexnet(num_classes=10)

# Sending model to device
net.to(device)
print(summary(net,(3,227,227))) # visualize number of parameters' net, output of each layer and total mega bytes necessary for forward pass
                                # and stored weights. 

# função de custo (ou loss)
loss = nn.CrossEntropyLoss()

# carregamento do dado: mnist
train_iter, test_iter = load_data_cifar10(batch_size, resize=227)

# trainer do gluon
trainer = optim.SGD(net.parameters(), lr=lr, weight_decay=wd_lambda, momentum=0.9)

# treinamento e validação via Pytorch
train_validate(net, train_iter, test_iter, batch_size, trainer, loss, 
                num_epochs)

0it [00:00, ?it/s]

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 56, 56]          23,296
              ReLU-2           [-1, 64, 56, 56]               0
         MaxPool2d-3           [-1, 64, 27, 27]               0
            Conv2d-4          [-1, 192, 27, 27]         307,392
              ReLU-5          [-1, 192, 27, 27]               0
         MaxPool2d-6          [-1, 192, 13, 13]               0
            Conv2d-7          [-1, 384, 13, 13]         663,936
              ReLU-8          [-1, 384, 13, 13]               0
            Conv2d-9          [-1, 256, 13, 13]         884,992
             ReLU-10          [-1, 256, 13, 13]               0
           Conv2d-11          [-1, 256, 13, 13]         590,080
             ReLU-12          [-1, 256, 13, 13]               0
        MaxPool2d-13            [-1, 256, 6, 6]               0
AdaptiveAvgPool2d-14            [-1, 25

170500096it [00:03, 42858845.28it/s]                               


Extracting /root/.pytorch/datasets/fashion-mnist/cifar-10-python.tar.gz to /root/.pytorch/datasets/fashion-mnist
Files already downloaded and verified
training on cuda
epoch 1, train loss 2.2150, train acc 0.160, test loss 1.9850, test acc 0.262, time 128.2 sec
epoch 2, train loss 1.7934, train acc 0.332, test loss 1.5451, test acc 0.427, time 128.0 sec
epoch 3, train loss 1.4409, train acc 0.475, test loss 1.3463, test acc 0.514, time 128.5 sec
epoch 4, train loss 1.1734, train acc 0.579, test loss 1.1015, test acc 0.608, time 128.3 sec
epoch 5, train loss 0.9665, train acc 0.662, test loss 0.9602, test acc 0.664, time 129.0 sec
epoch 6, train loss 0.8200, train acc 0.714, test loss 0.8959, test acc 0.683, time 128.5 sec
epoch 7, train loss 0.7055, train acc 0.752, test loss 0.7173, test acc 0.756, time 129.0 sec
epoch 8, train loss 0.6192, train acc 0.785, test loss 0.7065, test acc 0.763, time 129.4 sec
epoch 9, train loss 0.5458, train acc 0.810, test loss 0.6812, test acc 0.769, t

KeyboardInterrupt: ignored

## Extrator de características

A terceira e última estratégia, mostrada na figura abaixo, é usar uma rede neural pré-treinada em algum dataset grande para extrair características de um outro dataset. Essa estratégia é preferível quando o dataset que se quer extrair as *features* tem muito poucas amostras, inviabilizando o treinamento ou *fine-tuning* da rede.

<p align="center">
  <img width=600 src="https://drive.google.com/uc?export=view&id=1pWGfQIAeOODIvm-IQ7De4kl60XpRYbb5">
</p>

Existem duas formas de se explorar essa estratégia. A primeira consiste em substituir e treinar somente a última camada da rede neural. Nessa primeira forma, todas as outras camadas da rede ficam com *learning rate* 0, ou seja, não aprendem nada, e são somente usadas como codificadores/extratores de características. A segunda forma, *features* das imagens do dataset que se quer classificar são extraídas da penúltima camada da rede pré-treinada (geralmente, a camada antes da camada de classificação). Essas *features* são então usadas para se treinar um agoritmo externo (como um SVM ou *random forest*), que então classifica o dataset.

In [0]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [38]:

# rede baseada na AlexNet-5 
net = torchvision.models.alexnet(pretrained=True)

# Sending model to device
net.to(device)


feature_extract = True
set_parameter_requires_grad(net, feature_extract) # Iremos treinar somente a última camada

print(summary(net,(3,227,227))) # visualize number of parameters' net, output of each layer and total mega bytes necessary for forward pass
                                # and stored weights. 

num_ftrs = net.classifier[6].in_features
net.classifier[6] = nn.Linear(num_ftrs,10) # Alterando a última layer para retornar 10 classes ao invés de 1000

# Sending model to device
net.to(device)

# Verifique no output a última camada do classifier, podemos ver que sua saída é 10
print(net)

# Podemos ver que este output mostra que apenas  40970 parâmetros serão treinados. Ou seja, somente a última camada
print(summary(net,(3,227,227))) 

# Código retirado de https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html

# Gather the parameters to be optimized/updated in this run. If we are
#  finetuning we will be updating all parameters. However, if we are
#  doing feature extract method, we will only update the parameters
#  that we have just initialized, i.e. the parameters with requires_grad
#  is True.
params_to_update = net.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in net.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in net.named_parameters():
        if param.requires_grad == True:
            print("\t",name)


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 56, 56]          23,296
              ReLU-2           [-1, 64, 56, 56]               0
         MaxPool2d-3           [-1, 64, 27, 27]               0
            Conv2d-4          [-1, 192, 27, 27]         307,392
              ReLU-5          [-1, 192, 27, 27]               0
         MaxPool2d-6          [-1, 192, 13, 13]               0
            Conv2d-7          [-1, 384, 13, 13]         663,936
              ReLU-8          [-1, 384, 13, 13]               0
            Conv2d-9          [-1, 256, 13, 13]         884,992
             ReLU-10          [-1, 256, 13, 13]               0
           Conv2d-11          [-1, 256, 13, 13]         590,080
             ReLU-12          [-1, 256, 13, 13]               0
        MaxPool2d-13            [-1, 256, 6, 6]               0
AdaptiveAvgPool2d-14            [-1, 25

In [39]:
# Treinando a última camada da rede acima
# parâmetros: número de epochs, learning rate (ou taxa de aprendizado), 
# tamanho do batch, e lambda do weight decay
num_epochs, lr, batch_size, wd_lambda = 20, 0.001, 100, 0.0001


# função de custo (ou loss)
loss = nn.CrossEntropyLoss()

# carregamento do dado: mnist
train_iter, test_iter = load_data_cifar10(batch_size, resize=227)

# trainer do gluon
trainer = optim.SGD(params_to_update, lr=lr, weight_decay=wd_lambda, momentum=0.9)

# treinamento e validação via Pytorch
train_validate(net, train_iter, test_iter, batch_size, trainer, loss, 
                num_epochs)

Files already downloaded and verified
Files already downloaded and verified
training on cuda
epoch 1, train loss 1.3217, train acc 0.534, test loss 1.1989, test acc 0.581, time 87.6 sec
epoch 2, train loss 1.1632, train acc 0.591, test loss 1.1641, test acc 0.582, time 87.6 sec
epoch 3, train loss 1.1363, train acc 0.599, test loss 1.1220, test acc 0.604, time 86.9 sec
epoch 4, train loss 1.1119, train acc 0.605, test loss 1.0966, test acc 0.609, time 86.1 sec
epoch 5, train loss 1.0952, train acc 0.614, test loss 1.0971, test acc 0.609, time 85.5 sec
epoch 6, train loss 1.0879, train acc 0.615, test loss 1.1102, test acc 0.605, time 85.1 sec
epoch 7, train loss 1.0819, train acc 0.619, test loss 1.0763, test acc 0.619, time 86.7 sec
epoch 8, train loss 1.0756, train acc 0.619, test loss 1.0926, test acc 0.610, time 87.2 sec
epoch 9, train loss 1.0718, train acc 0.621, test loss 1.0838, test acc 0.616, time 86.3 sec
epoch 10, train loss 1.0624, train acc 0.624, test loss 1.0802, test a

KeyboardInterrupt: ignored

In [33]:
# parâmetros: número de epochs, learning rate (ou taxa de aprendizado), 
# tamanho do batch, e lambda do weight decay
num_epochs, lr, batch_size, wd_lambda = 20, 0.01, 100, 0.0001

# rede baseada na AlexNet-5 
net = torchvision.models.alexnet(pretrained=True)

# Sending model to device
net.to(device)

# carregamento do dado: fashion mnist
train_iter, test_iter = load_data_cifar10(batch_size, resize=227)  

# remove last fully-connected layer
new_classifier = nn.Sequential(*list(net.classifier.children())[:-1])
net.classifier = new_classifier

print(summary(net,(3,227,227))) # visualize number of parameters' net, output of each layer and total mega bytes necessary for forward pass
                                # and stored weights. 

first = True    
with torch.no_grad():
    for X, y in train_iter:
        X, y = X.to(device), y.to(device)
        features = net(X)
        if first is True:
          train_features = features.cpu().numpy()
          train_labels = y.cpu().numpy()
          first = False
        else:
          train_features = np.concatenate((train_features, features.cpu().numpy()))
          train_labels = np.concatenate((train_labels, y.cpu().numpy()))


first = True    
with torch.no_grad():
    for X, y in test_iter:
        X, y = X.to(device), y.to(device)
        features = net(X)
        if first is True:
          test_features = features.cpu().numpy()
          test_labels = y.cpu().numpy()
          first = False
        else:
          test_features = np.concatenate((test_features, features.cpu().numpy()))
          test_labels = np.concatenate((test_labels, y.cpu().numpy()))

          
print(train_features.shape, test_features.shape)

Files already downloaded and verified
Files already downloaded and verified
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 56, 56]          23,296
              ReLU-2           [-1, 64, 56, 56]               0
         MaxPool2d-3           [-1, 64, 27, 27]               0
            Conv2d-4          [-1, 192, 27, 27]         307,392
              ReLU-5          [-1, 192, 27, 27]               0
         MaxPool2d-6          [-1, 192, 13, 13]               0
            Conv2d-7          [-1, 384, 13, 13]         663,936
              ReLU-8          [-1, 384, 13, 13]               0
            Conv2d-9          [-1, 256, 13, 13]         884,992
             ReLU-10          [-1, 256, 13, 13]               0
           Conv2d-11          [-1, 256, 13, 13]         590,080
             ReLU-12          [-1, 256, 13, 13]               0
        MaxPool2d-13       

In [34]:
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

clf = LinearSVC()
clf.fit(train_features, train_labels)

pred = clf.predict(test_features)
print(accuracy_score(test_labels, pred))

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.5272


## *Fine-tuning*

A segunda estratégia é chamada de *fine-tuning*, e é comumente classificada como um estratégia de *transfer learning*, onde o aprendizado é transferido entre datasets.
Especificamente, esta estratégia, representada na figura abaixo, tenta usar um modelo pré-treinado aprendido anteriormente em algum dataset (geralmente muito grande, como o [ImageNet](http://www.image-net.org/)) para classificar outro conjunto de dados diferentes (geralmente com poucas amostras).

<p align="center">
  <img width=600 src="https://drive.google.com/uc?export=view&id=1CoOfpMcQAEl9YAL0lgW11LLYpDcnL4dQ">
</p>

Como esses dados podem possuir características diferentes, treinamos a rede usando um *learning rate* pequeno, apenas para fazer pequenos ajustes nos pesos. Entretanto, como esses datasets geralmente tem número e classes diferentes, a última camada não é usada nessa transferência de peso e, geralmente, é inicializada aleatoriamente (e por isso, tem um *learning rate* mais alto que as demais camadas).

Por fim, é um [fato conhecido](https://arxiv.org/pdf/1602.01517.pdf) que as redes neurais conseguem aprender características de baixo nível nas camadas iniciais. Geralmente, essas características são comuns à vários datasets. Por isso, uma opção durante o processo de *fine-tuning* é "congelar" as camadas iniciais (ou seja, não treiná-las) e treinar somente as demais camadas com taxa de aprendizado bem pequeno (exceto pela camada de classificação).

No bloco de código abaixo, importamos a rede pré-treinada [AlexNet](https://papers.nips.cc/paper/4824-imagenet-classification-with-deep-convolutional-neural-networks.pdf), que foi treinada no dataset do [ImageNet](http://www.image-net.org/), que tem 1000 classes. Como iremos fazer *fine-tuning* nessa arquitetura para o dataset do [CIFAR-10](https://www.cs.toronto.edu/~kriz/cifar.html), que tem somente 10 classes, removeremos a última camada e criaremos uma nova camada inicializada aleatoriamente.

In [37]:

# rede baseada na AlexNet-5 
net = torchvision.models.alexnet(pretrained=True)

# Sending model to device
net.to(device)

print(summary(net,(3,227,227))) # visualize number of parameters' net, output of each layer and total mega bytes necessary for forward pass
                                # and stored weights. 

num_ftrs = net.classifier[6].in_features
net.classifier[6] = nn.Linear(num_ftrs,10) # Alterando a última layer para retornar 10 classes ao invés de 1000

# Sending model to device
net.to(device)

# Verifique no output a última camada do classifier, podemos ver que sua saída é 10
print(net)

# Podemos ver que este output mostra que apenas  40970 parâmetros serão treinados. Ou seja, somente a última camada
print(summary(net,(3,227,227))) 

# Treinando a última camada da rede acima
# parâmetros: número de epochs, learning rate (ou taxa de aprendizado), 
# tamanho do batch, e lambda do weight decay
num_epochs, lr, batch_size, wd_lambda = 20, 0.001, 100, 0.0001


# função de custo (ou loss)
loss = nn.CrossEntropyLoss()

# carregamento do dado: mnist
train_iter, test_iter = load_data_cifar10(batch_size, resize=227)

# trainer do gluon
trainer = optim.SGD([
                {'params': net.features.parameters(), 'lr': lr * 0.1},
                {'params': net.classifier[0:6].parameters(), 'lr': lr * 0.1},
                {'params': net.classifier[6].parameters(), 'lr': lr}], weight_decay=wd_lambda, momentum=0.9)

# treinamento e validação via Pytorch
train_validate(net, train_iter, test_iter, batch_size, trainer, loss, 
                num_epochs)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 56, 56]          23,296
              ReLU-2           [-1, 64, 56, 56]               0
         MaxPool2d-3           [-1, 64, 27, 27]               0
            Conv2d-4          [-1, 192, 27, 27]         307,392
              ReLU-5          [-1, 192, 27, 27]               0
         MaxPool2d-6          [-1, 192, 13, 13]               0
            Conv2d-7          [-1, 384, 13, 13]         663,936
              ReLU-8          [-1, 384, 13, 13]               0
            Conv2d-9          [-1, 256, 13, 13]         884,992
             ReLU-10          [-1, 256, 13, 13]               0
           Conv2d-11          [-1, 256, 13, 13]         590,080
             ReLU-12          [-1, 256, 13, 13]               0
        MaxPool2d-13            [-1, 256, 6, 6]               0
AdaptiveAvgPool2d-14            [-1, 25

## Prática

1. É possível melhorar o resultado obtido anteriormente?
Estude o [model_zoo](https://pytorch.org/docs/stable/torchvision/models.html)  e tente usar as estratégias anteriores com diferentes redes neurais para melhorar o resultado.
Algumas redes possíveis:

- [MobileNets](https://arxiv.org/abs/1801.04381)
- [VGGs](https://arxiv.org/abs/1409.1556)
- [ResNets](https://arxiv.org/abs/1603.05027)
- [DenseNets](https://arxiv.org/pdf/1608.06993.pdf)

2. Procure agora congelar algumas camadas para realizar o *fine-tuning*. Essa estratégia é melhor quando se tem poucas imagens para fazer o *fine-tuning*.

3. Procura usar outros algoritmos de aprendizado de máquina (como [*random forest*](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) e [SVM-RBF](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html)) para classificar *deep features* extraídas de uma rede neural pré-treinada.
  1. Procure também extrair e classificar *features* de outras camadas convolucionais.

4. Procure usar as diferentes estratégias para melhorar os resultados dos datasets que já usamos, como [MNIST](https://pytorch.org/docs/stable/torchvision/datasets.html#torchvision.datasets.MNIST) e [Fashion MNIST](https://pytorch.org/docs/stable/torchvision/datasets.html#torchvision.datasets.FashionMNIST).